In [147]:
# -------------------------------------------------------
# DAY 1: NLP BASICS + SENTIMENT ANALYSIS MODEL
# -------------------------------------------------------
# Notebook by: <your name>
# Goal: Learn NLP basics and build a sentiment analysis model
# -------------------------------------------------------

In [150]:
# NLP Summary
#- Definition:NLP stands for Natural Language Processing, which is a branch of Data Science and it enables machines to understand and interpret human language.
#- What NLP can do: It can do tasks such as speech recognition, language translation, chatbots and virtual assistants, text summarization, information retrieval and sentiment analysis
#- Real-world examples: Some of the daily use examples include google, siri, google translation, chatgpt and so on.

In [152]:
### Preprocessing Summary
#- Tokenization: It is one of the major step in text preprocessing which splits the text into tokens such as words or sentences
#- Stopwords: Stopwords are the most commonly used/repeated words which won't play that major role in understanding the text, which might create noise so should be removed
#- Stemming vs Lemmatization: Both are used to convert the words to their base/root form but stemming won't considers the meaning/context of root form it just stems irrespective of their context.
#  For example crazy word will be stemmed into crazi which irrelevant, whereas lemmatization follows a predefined words of root forms so it stems the different forms of words to original or root form to understand the meaning.
#- Bag of Words: It is a method of feature extraction which takes the documents as units which consists of bag of words and converts to vector form where words with their frequency(count) will present.
#- TF-IDF: unlike BOW, where all the words are treated equally, but in some cases some particular words may repeat more often than usual due to their text/topic so this method first creates document frequency then it divides the term frequency by the document frequency. 

In [155]:
#Data Loading
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\kunar\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [156]:
movie_reviews.categories()

['neg', 'pos']

In [157]:
len(movie_reviews.fileids('pos')), len(movie_reviews.fileids('neg'))

(1000, 1000)

In [158]:
import pandas as pd

data = []

for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        text = movie_reviews.raw(fileid)
        data.append((text, category))

df = pd.DataFrame(data, columns=['text', 'label'])
df.head()

text  \
0  plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat's the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it's simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no idea what's going on . \nthere are dreams , there are characters coming back from the dead , there are others who look like the dead , there are strange apparitions , there are disappearances , there are a looooot of chase scenes , there are tons of weird things that happen , and most of it is simply not explained . \nnow i personally don't mind trying to unravel a film every now and then , but when all it does is give me the same clue over and over again , i get kind of fed up after a while , which is this film's biggest problem . \nit's obviously got this big secret to hide , but it seems to want to hide it completely until its final five minutes . \nand do they make things entertaining , thrilling or even engaging , in the meantime ? \nnot really . \nthe sad part is that the arrow and i both dig on flicks like this , so we actually figured most of it out by the half-way point , so all of the strangeness after that did start to make a little bit of sense , but it still didn't the make the film all that more entertaining . \ni guess the bottom line with movies like this is that you should always make sure that the audience is " into it " even before they are given the secret password to enter your world of understanding . \ni mean , showing melissa sagemiller running away from visions for about 20 minutes throughout the movie is just plain lazy ! ! \nokay , we get it . . . there \nare people chasing her and we don't know who they are . \ndo we really need to see it over and over again ? \nhow about giving us different scenes offering further insight into all of the strangeness going down in the movie ? \napparently , the studio took this film away from its director and chopped it up themselves , and it shows . \nthere might've been a pretty decent teen mind-fuck movie in here somewhere , but i guess " the suits " decided that turning it into a music video with little edge , would make more sense . \nthe actors are pretty good for the most part , although wes bentley just seemed to be playing the exact same character that he did in american beauty , only in a new neighborhood . \nbut my biggest kudos go out to sagemiller , who holds her own throughout the entire film , and actually has you feeling her character's unraveling . \noverall , the film doesn't stick because it doesn't entertain , it's confusing , it rarely excites and it feels pretty redundant for most of its runtime , despite a pretty cool ending and explanation to all of the craziness that came before it . \noh , and by the way , this is not a horror or teen slasher flick . . . it's \njust packaged to look that way because someone is apparently assuming that the genre is still hot with the kids . \nit also wrapped production two years ago and has been sitting on the shelves ever since . \nwhatever . . . skip \nit ! \nwhere's joblo coming from ? \na nightmare of elm street 3 ( 7/10 ) - blair witch 2 ( 7/10 ) - the crow ( 9/10 ) - the crow : salvation ( 4/10 ) - lost

In [159]:
df['label'] = df['label'].map({'neg': 0, 'pos': 1})

In [160]:
df.shape

(2000, 2)

In [161]:
#Text Preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = text.lower()
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(tokens)

df['clean_text'] = df['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kunar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kunar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [162]:
#Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

In [163]:
#Modelling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [164]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


In [165]:
#Evaluation
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.845
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       199
           1       0.83      0.87      0.85       201

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400



In [166]:
#prediction
print(model.predict(vectorizer.transform([
    "I loved this movie!",
    "This was the worst movie ever.",
    "Not bad but not great either."
])))

[1 0 0]


In [167]:
#Another Prediction
test = [
    "good", "bad", "love", "hate", "terrible", 
    "amazing", "this is bad", "this is good"
]

print(model.predict(vectorizer.transform(test)))


[1 0 1 1 0 1 0 1]
